In [11]:
import pandas as pd
import numpy as np
import re

tx = pd.read_excel("data/raw/QVI_transaction_data.xlsx")
pb = pd.read_excel("data/raw/QVI_purchase_behaviour.xlsx")

In [12]:
# Basic info
print("=== TRANSACTION DATA ===")
print(tx.shape)
print(tx.info())
print("\nSample rows:\n", tx.head(3))

print("\n=== PURCHASE BEHAVIOUR DATA ===")
print(pb.shape)
print(pb.info())
print("\nSample rows:\n", pb.head(3))


=== TRANSACTION DATA ===
(264836, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DATE            264836 non-null  int64  
 1   STORE_NBR       264836 non-null  int64  
 2   LYLTY_CARD_NBR  264836 non-null  int64  
 3   TXN_ID          264836 non-null  int64  
 4   PROD_NBR        264836 non-null  int64  
 5   PROD_NAME       264836 non-null  object 
 6   PROD_QTY        264836 non-null  int64  
 7   TOT_SALES       264836 non-null  float64
dtypes: float64(1), int64(6), object(1)
memory usage: 16.2+ MB
None

Sample rows:
     DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR  \
0  43390          1            1000       1         5   
1  43599          1            1307     348        66   
2  43605          1            1343     383        61   

                                PROD_NAME  PROD_QTY  TOT_SALES  
0  Natural Chip 

In [13]:
#  Quick numeric overview
print("\n=== Numeric Summary (Transactions) ===")
print(tx.describe().T[["mean", "std", "min", "max"]].head(10))



=== Numeric Summary (Transactions) ===
                         mean           std      min        max
DATE             43464.036260    105.389282  43282.0    43646.0
STORE_NBR          135.080110     76.784180      1.0      272.0
LYLTY_CARD_NBR  135549.476404  80579.978022   1000.0  2373711.0
TXN_ID          135158.310815  78133.026026      1.0  2415841.0
PROD_NBR            56.583157     32.826638      1.0      114.0
PROD_QTY             1.907309      0.643654      1.0      200.0
TOT_SALES            7.304200      3.083226      1.5      650.0


In [14]:
# --- Data Quality Check: DATE ---
print(tx["DATE"].head(10).tolist())
raw2 = pd.read_excel("data/raw/QVI_transaction_data.xlsx")
print(type(raw2.loc[0,"DATE"]), raw2.loc[0,"DATE"])


[43390, 43599, 43605, 43329, 43330, 43604, 43601, 43601, 43332, 43330]
<class 'numpy.int64'> 43390


In [15]:
# Force to numeric if the column was auto-parsed to datetime improperly
s = pd.to_numeric(tx["DATE"], errors="coerce")

# Helper to convert mixed encodings safely:
# - Excel serial days   ~ 30000..50000 (≈ 1982–2137; Quantium ~ 43k)
# - Unix seconds        ~ 1.0e9..2.0e9
# - Unix milliseconds   ~ 1.0e12..2.0e12
# Otherwise -> NaT
def to_datetime_mixed(v):
    if pd.isna(v):
        return pd.NaT
    try:
        v = float(v)
    except Exception:
        return pd.NaT
    if 30000 <= v <= 50000:
        return pd.to_datetime("1899-12-30") + pd.to_timedelta(int(v), unit="D")
    if 1_000_000_000 <= v <= 2_000_000_000:
        return pd.to_datetime(int(v), unit="s")
    if 1_000_000_000_000 <= v <= 2_000_000_000_000:
        return pd.to_datetime(int(v), unit="ms")
    return pd.NaT  # out-of-range or noise

tx["DATE"] = s.apply(to_datetime_mixed)

# Drop invalid/missing dates
tx = tx.dropna(subset=["DATE"]).copy()

# Normalize to date (remove time) and quick sanity
tx["DATE"] = tx["DATE"].dt.floor("D")
print("Date range:", tx["DATE"].min().date(), "→", tx["DATE"].max().date())
print("Rows with valid DATE:", len(tx))
print("\n Sample dates:", sorted(tx["DATE"].sample(min(10, len(tx)), random_state=42).tolist()))


Date range: 2018-07-01 → 2019-06-30
Rows with valid DATE: 264836

 Sample dates: [Timestamp('2018-07-14 00:00:00'), Timestamp('2018-07-21 00:00:00'), Timestamp('2018-09-15 00:00:00'), Timestamp('2018-09-22 00:00:00'), Timestamp('2018-10-18 00:00:00'), Timestamp('2019-02-19 00:00:00'), Timestamp('2019-03-09 00:00:00'), Timestamp('2019-04-02 00:00:00'), Timestamp('2019-04-09 00:00:00'), Timestamp('2019-06-12 00:00:00')]


In [16]:
# --- Data Quality Check: STORE_NBR (Store Number) ---
# Check datatype and basic structure
print(" Column dtype:", tx["STORE_NBR"].dtype)
print(" Unique store count:", tx["STORE_NBR"].nunique())
print(" Sample store numbers:", sorted(tx['STORE_NBR'].unique()[:10]))

# Check for missing or invalid values
null_count = tx["STORE_NBR"].isna().sum()
print(f"\nMissing STORE_NBR values: {null_count} ({null_count / len(tx) * 100:.3f}%)")

# Check numeric validity (should be integer, positive)
invalid_values = tx.loc[
    (tx["STORE_NBR"] < 0) | (tx["STORE_NBR"] % 1 != 0), "STORE_NBR"
]
if not invalid_values.empty:
    print("\n Invalid STORE_NBR values detected (negative or non-integer):")
    print(invalid_values.unique()[:10])
else:
    print("\n All STORE_NBR values are positive integers.")

# Range sanity check
min_store = tx["STORE_NBR"].min()
max_store = tx["STORE_NBR"].max()
print(f"\n STORE_NBR range: {min_store} → {max_store}")

 Column dtype: int64
 Unique store count: 272
 Sample store numbers: [np.int64(1), np.int64(2), np.int64(4), np.int64(5), np.int64(7), np.int64(8), np.int64(9), np.int64(13), np.int64(19), np.int64(20)]

Missing STORE_NBR values: 0 (0.000%)

 All STORE_NBR values are positive integers.

 STORE_NBR range: 1 → 272


In [17]:
# --- Data Quality Check: LYLTY_CARD_NBR (Customer ID) ---
# Check datatype and missing values
print(" Column dtype:", tx["LYLTY_CARD_NBR"].dtype)
null_count = tx["LYLTY_CARD_NBR"].isna().sum()
print(f" Missing LYLTY_CARD_NBR values: {null_count} ({null_count / len(tx) * 100:.3f}%)")

# Drop nulls if any
if null_count > 0:
    tx = tx.dropna(subset=["LYLTY_CARD_NBR"]).copy()
    print(" Null values removed from LYLTY_CARD_NBR.")
else:
    print(" No nulls to remove in LYLTY_CARD_NBR.")

# Ensure numeric and positive
tx["LYLTY_CARD_NBR"] = pd.to_numeric(tx["LYLTY_CARD_NBR"], errors="coerce")
invalid_ids = tx.loc[tx["LYLTY_CARD_NBR"] <= 0, "LYLTY_CARD_NBR"]
if not invalid_ids.empty:
    print("\n Invalid (non-positive) customer IDs detected:")
    print(invalid_ids.unique()[:10])
else:
    print("\n All customer IDs are positive integers.")

# Check for duplicate transactions by same customer on same date
dupes = tx[tx.duplicated(subset=["LYLTY_CARD_NBR", "DATE"], keep=False)]

if not dupes.empty:
    print(f"\n Potential duplicate transactions found: {len(dupes)} rows")
    print("Sample duplicate records:")
    print(dupes.head(5)[["LYLTY_CARD_NBR", "DATE", "TXN_ID", "TOT_SALES"]])
else:
    print("\n No duplicate customer transactions found on the same date.")

 Column dtype: int64
 Missing LYLTY_CARD_NBR values: 0 (0.000%)
 No nulls to remove in LYLTY_CARD_NBR.

 All customer IDs are positive integers.

 Potential duplicate transactions found: 3410 rows
Sample duplicate records:
     LYLTY_CARD_NBR       DATE  TXN_ID  TOT_SALES
41            55073 2019-05-20   48887       3.25
42            55073 2019-05-20   48887       4.60
376            7364 2019-01-10    7739       8.80
377            7364 2019-01-10    7739      11.40
418           12301 2018-10-18   10982       8.80


In [18]:
# Check if duplicates also share the same TXN_ID
same_txn = dupes.groupby(["LYLTY_CARD_NBR", "DATE"])["TXN_ID"].nunique().reset_index()
same_txn_issue = same_txn[same_txn["TXN_ID"] > 1]

print(f" Duplicates with different TXN_IDs: {len(same_txn_issue)}")

if len(same_txn_issue) == 0:
    print(" All duplicates belong to the same TXN_ID → multi-product purchases (not true duplicates).")
else:
    print(" Some duplicates have different TXN_IDs → potential data duplication.")
    print(same_txn_issue.head(10))


 Duplicates with different TXN_IDs: 3
 Some duplicates have different TXN_IDs → potential data duplication.
     LYLTY_CARD_NBR       DATE  TXN_ID
985          155014 2019-02-22       3
986          155035 2018-09-17       2
989          155092 2018-12-16       2


In [19]:
# --- Data Quality Check: PROD_NAME (Product Name) ---

# Compile regex patterns
re_pack = re.compile(r"(\d+)\s*[Gg]\b")  # e.g. 175g, 210G

#  multi-word brands first to avoid partial matches
multi_brands = [
    r"RED ROCK DELI", r"NATURAL CHIP CO", r"GRAIN WAVES", r"OLD EL PASO"
]
re_multi_brand = re.compile(rf"^({'|'.join(multi_brands)})\b", re.I)
re_first_token = re.compile(r"^([A-Za-z']+)\b")

# --- expanded variant line patterns ---
re_variant = re.compile(
    r"\b("
    r"CRINKLE\s+CUT|THIN(LY)?\s+CUT|RIDGE(S)?|RIDGE\s+CUT|WAVES?|WAVY|"
    r"CORN\s+CHIPS?|TORTILLA|SENSATIONS|PAPADUMS|CRIS?PS?"
    r")\b",
    re.I
)

# --- brand normalization map ---
brand_alias = {
    "RRD": "Red Rock Deli",
    "RED ROCK": "Red Rock Deli",
    "RED ROCK DELI": "Red Rock Deli",
    "WW": "Woolworths",
    "WOOLWORTHS": "Woolworths",
    "DORITO": "Doritos",
    "DORITOS": "Doritos",
    "INFZNS": "Infuzions",
    "INFUZIONS": "Infuzions",
    "SMITH": "Smiths",
    "SMITHS": "Smiths",
    "NATURAL CHIP": "Natural Chip Co",
    "NATURAL CHIP CO": "Natural Chip Co",
    "GRAIN WAVES": "Grain Waves",
}

# --- helpers ---
def normalize_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def normalize_brand(b: str) -> str:
    if not b:
        return b
    b_up = b.upper().strip()
    for k, v in brand_alias.items():
        if b_up.startswith(k):
            return v
    return b.title().strip()

def extract_parts(name: str):
    """Return BRAND, VARIANT_LINE, FLAVOR, PACK_SIZE from PROD_NAME."""
    if pd.isna(name):
        return pd.Series([None, None, None, None],
                         index=["BRAND","VARIANT_LINE","FLAVOR","PACK_SIZE"])

    raw = normalize_spaces(str(name))

    # ---- PACK_SIZE ----
    pack_size = None
    remainder = raw
    m_pack = re_pack.search(raw)
    if m_pack:
        pack_size = float(m_pack.group(1))
        remainder = normalize_spaces(re_pack.sub("", remainder))

    # ---- BRAND ----
    brand = None
    rem_after_brand = remainder

    m_mb = re_multi_brand.search(remainder.upper())
    if m_mb:
        start, end = m_mb.span()
        brand = remainder[start:end].strip()
        rem_after_brand = remainder[end:].strip()
    else:
        m1 = re_first_token.search(remainder)
        if m1:
            brand = m1.group(1).strip()
            rem_after_brand = remainder[m1.end():].strip()

    brand = normalize_brand(brand)

    # ---- VARIANT_LINE ----
    variant_line = None
    rem_after_variant = rem_after_brand
    if rem_after_brand:
        m_v = re_variant.search(rem_after_brand)
        if m_v:
            variant_line = m_v.group(1).strip()
            rem_after_variant = normalize_spaces(
                re_variant.sub("", rem_after_brand, count=1)
            )

    # ---- FLAVOR ----
    flavor = rem_after_variant if rem_after_variant else None
    if flavor:
        flavor = normalize_spaces(flavor)
        if not flavor.isupper():
            flavor = flavor.title()

    return pd.Series([brand, variant_line, flavor, pack_size],
                     index=["BRAND","VARIANT_LINE","FLAVOR","PACK_SIZE"])


# Apply to dataframe
parsed = tx["PROD_NAME"].apply(extract_parts)
tx[["BRAND","VARIANT_LINE","FLAVOR","PACK_SIZE"]] = parsed

# Quick diagnostics
print(" Created columns: BRAND, VARIANT_LINE, FLAVOR, PACK_SIZE")

print("\n Null counts:")
print(tx[["BRAND","VARIANT_LINE","FLAVOR","PACK_SIZE"]].isna().sum())

print("\n Top 15 brands:")
print(tx["BRAND"].value_counts().head(15).to_string())

print("\n Top variant lines:")
print(tx["VARIANT_LINE"].dropna().value_counts().head(15).to_string())

print("\n PACK_SIZE basic stats:")
print(tx["PACK_SIZE"].describe())


 Created columns: BRAND, VARIANT_LINE, FLAVOR, PACK_SIZE

 Null counts:
BRAND                0
VARIANT_LINE    188977
FLAVOR               0
PACK_SIZE            0
dtype: int64

 Top 15 brands:
BRAND
Kettle           41288
Smiths           31823
Doritos          28147
Pringles         25102
Red Rock Deli    17779
Woolworths       14757
Infuzions        14201
Thins            14075
Cobs              9693
Tostitos          9471
Twisties          9454
Old El Paso       9324
Tyrrells          6442
Grain Waves       6272
Cheezels          4603

 Top variant lines:
VARIANT_LINE
Crinkle Cut    17621
Crisps         12607
Corn Chips     12502
Tortilla        9580
Sensations      9429
Corn Chip       6376
Thinly Cut      3133
Crips           3104
Papadums        1507

 PACK_SIZE basic stats:
count    264836.000000
mean        182.427004
std          64.327196
min          70.000000
25%         150.000000
50%         170.000000
75%         175.000000
max         380.000000
Name: PACK_SIZE, dtype:

In [20]:
# --- Data Quality Check: QUANTITY (PROD_QTY) ---

# Summary statistics
print(" Summary statistics for QUANTITY:")
print(tx["PROD_QTY"].describe())

# Check for missing, zero, or negative values
print("\n Missing, zero, and negative value check:")
print(f"Null values: {tx['PROD_QTY'].isna().sum()}")
print(f"Zero values: {(tx['PROD_QTY'] == 0).sum()}")
print(f"Negative values: {(tx['PROD_QTY'] < 0).sum()}")

# Frequency of unique values to understand purchase size patterns
print("\n Frequency of common purchase quantities:")
print(tx["PROD_QTY"].value_counts().sort_index())

#  Identify potential outliers (very large purchase quantities)
print("\n Potential outliers (QUANTITY > 10):")
print(tx[tx["PROD_QTY"] > 10][["PROD_NAME", "PROD_QTY"]].head(10))


 Summary statistics for QUANTITY:
count    264836.000000
mean          1.907309
std           0.643654
min           1.000000
25%           2.000000
50%           2.000000
75%           2.000000
max         200.000000
Name: PROD_QTY, dtype: float64

 Missing, zero, and negative value check:
Null values: 0
Zero values: 0
Negative values: 0

 Frequency of common purchase quantities:
PROD_QTY
1       27518
2      236039
3         430
4         397
5         450
200         2
Name: count, dtype: int64

 Potential outliers (QUANTITY > 10):
                              PROD_NAME  PROD_QTY
69762  Dorito Corn Chp     Supreme 380g       200
69763  Dorito Corn Chp     Supreme 380g       200


In [21]:
# Remove unrealistic outliers in quantity (e.g., 200 packs)
tx = tx[tx["PROD_QTY"] <= 10].copy()

print(" Cleaned QUANTITY column.")
print(tx["PROD_QTY"].describe())
print(tx["PROD_QTY"].value_counts().sort_index())


 Cleaned QUANTITY column.
count    264834.000000
mean          1.905813
std           0.343436
min           1.000000
25%           2.000000
50%           2.000000
75%           2.000000
max           5.000000
Name: PROD_QTY, dtype: float64
PROD_QTY
1     27518
2    236039
3       430
4       397
5       450
Name: count, dtype: int64


In [22]:
# --- Data Quality Check: TOT_SALES (Total Sales Amount) ---

#  Summary statistics
print(" Summary statistics for TOT_SALES:")
print(tx["TOT_SALES"].describe())

#  Check for missing, zero, or negative values
print("\n Missing, zero, and negative value check:")
print(f"Null values: {tx['TOT_SALES'].isna().sum()}")
print(f"Zero values: {(tx['TOT_SALES'] == 0).sum()}")
print(f"Negative values: {(tx['TOT_SALES'] < 0).sum()}")

#  Frequency of common sales amounts (rounded for better readability)
print("\n Frequency of common transaction amounts:")
print(tx["TOT_SALES"].round(2).value_counts().sort_index().head(20))

#  Identify potential outliers (very large total sales values)
print("\n Potential outliers (TOT_SALES > 30):")
print(tx[tx["TOT_SALES"] > 30][["PROD_NAME", "PROD_QTY", "TOT_SALES"]].head(10))


 Summary statistics for TOT_SALES:
count    264834.000000
mean          7.299346
std           2.527241
min           1.500000
25%           5.400000
50%           7.400000
75%           9.200000
max          29.500000
Name: TOT_SALES, dtype: float64

 Missing, zero, and negative value check:
Null values: 0
Zero values: 0
Negative values: 0

 Frequency of common transaction amounts:
TOT_SALES
1.50     354
1.70     708
1.80     175
1.90     930
2.10     732
2.30     186
2.40     182
2.60    1287
2.70    1231
2.80     189
2.90     886
3.00    5434
3.10     188
3.25      88
3.30    1539
3.40    5146
3.60    1963
3.70    2459
3.80    8585
3.90     601
Name: count, dtype: int64

 Potential outliers (TOT_SALES > 30):
Empty DataFrame
Columns: [PROD_NAME, PROD_QTY, TOT_SALES]
Index: []


In [23]:
# --- Data Quality Check: LYLTY_CARD_NBR (Customer ID) ---

# Summary info
print(" Summary statistics for LYLTY_CARD_NBR:")
print(pb["LYLTY_CARD_NBR"].describe())

# Check for missing or duplicate IDs
print("\n Missing and duplicate check:")
print(f"Null values: {pb['LYLTY_CARD_NBR'].isna().sum()}")
print(f"Duplicate values: {pb['LYLTY_CARD_NBR'].duplicated().sum()}")

# Inspect min and max to ensure numeric consistency
print("\n Range of customer IDs:")
print(f"Min ID: {pb['LYLTY_CARD_NBR'].min()}, Max ID: {pb['LYLTY_CARD_NBR'].max()}")

# Quick integrity cross-check with transaction data
common_ids = set(tx["LYLTY_CARD_NBR"]).intersection(set(pb["LYLTY_CARD_NBR"]))
print(f"\n Common customer IDs between datasets: {len(common_ids)}")
print(f"Unique IDs in transactions only: {tx['LYLTY_CARD_NBR'].nunique() - len(common_ids)}")
print(f"Unique IDs in purchase behaviour only: {pb['LYLTY_CARD_NBR'].nunique() - len(common_ids)}")


 Summary statistics for LYLTY_CARD_NBR:
count    7.263700e+04
mean     1.361859e+05
std      8.989293e+04
min      1.000000e+03
25%      6.620200e+04
50%      1.340400e+05
75%      2.033750e+05
max      2.373711e+06
Name: LYLTY_CARD_NBR, dtype: float64

 Missing and duplicate check:
Null values: 0
Duplicate values: 0

 Range of customer IDs:
Min ID: 1000, Max ID: 2373711

 Common customer IDs between datasets: 72636
Unique IDs in transactions only: 0
Unique IDs in purchase behaviour only: 1


In [24]:
# --- Data Quality Check: LIFESTAGE (Customer Life Stage) ---

# Summary info
print(" Summary of LIFESTAGE column:")
print(pb["LIFESTAGE"].describe())

# Check for missing or blank values
print("\n Missing or blank value check:")
print(f"Null values: {pb['LIFESTAGE'].isna().sum()}")
print(f"Blank strings: {(pb['LIFESTAGE'].str.strip() == '').sum()}")

# Unique category listing and counts
print("\n Unique life stage categories and counts:")
print(pb["LIFESTAGE"].value_counts().sort_index())

#  Detect potential typos or inconsistent formatting
print("\n Potential inconsistent capitalization or spacing:")
print(pb["LIFESTAGE"].unique())


 Summary of LIFESTAGE column:
count        72637
unique           7
top       RETIREES
freq         14805
Name: LIFESTAGE, dtype: object

 Missing or blank value check:
Null values: 0
Blank strings: 0

 Unique life stage categories and counts:
LIFESTAGE
MIDAGE SINGLES/COUPLES     7275
NEW FAMILIES               2549
OLDER FAMILIES             9780
OLDER SINGLES/COUPLES     14609
RETIREES                  14805
YOUNG FAMILIES             9178
YOUNG SINGLES/COUPLES     14441
Name: count, dtype: int64

 Potential inconsistent capitalization or spacing:
['YOUNG SINGLES/COUPLES' 'YOUNG FAMILIES' 'OLDER SINGLES/COUPLES'
 'MIDAGE SINGLES/COUPLES' 'NEW FAMILIES' 'OLDER FAMILIES' 'RETIREES']


In [26]:
# --- Data Quality Check: PREMIUM_CUSTOMER (Customer Price Segment) ---

# Summary info
print(" Summary of PREMIUM_CUSTOMER column:")
print(pb["PREMIUM_CUSTOMER"].describe())

# Check for missing or blank values
print("\n Missing or blank value check:")
print(f"Null values: {pb['PREMIUM_CUSTOMER'].isna().sum()}")
print(f"Blank strings: {(pb['PREMIUM_CUSTOMER'].str.strip() == '').sum()}")

# Unique category listing and counts
print("\n Unique customer type categories and counts:")
print(pb["PREMIUM_CUSTOMER"].value_counts().sort_index())

# Detect potential inconsistent capitalization or spacing
print("\n Potential inconsistent capitalization or spacing:")
print(pb["PREMIUM_CUSTOMER"].unique())


 Summary of PREMIUM_CUSTOMER column:
count          72637
unique             3
top       Mainstream
freq           29245
Name: PREMIUM_CUSTOMER, dtype: object

 Missing or blank value check:
Null values: 0
Blank strings: 0

 Unique customer type categories and counts:
PREMIUM_CUSTOMER
Budget        24470
Mainstream    29245
Premium       18922
Name: count, dtype: int64

 Potential inconsistent capitalization or spacing:
['Premium' 'Mainstream' 'Budget']


In [27]:
# --- Save cleaned datasets before merging ---

import os

# Define output directory (create if not exists)
output_dir = "data/cleaned"
os.makedirs(output_dir, exist_ok=True)

# Save cleaned transaction dataset
tx.to_csv(f"{output_dir}/QVI_transaction_cleaned.csv", index=False)
print(" Saved cleaned transaction data → data/cleaned/QVI_transaction_cleaned.csv")

# Save cleaned purchase behaviour dataset
pb.to_csv(f"{output_dir}/QVI_purchase_behaviour_cleaned.csv", index=False)
print(" Saved cleaned purchase behaviour data → data/cleaned/QVI_purchase_behaviour_cleaned.csv")

# Optional quick check: confirm file shapes and columns
print("\n Transaction Data Shape:", tx.shape)
print(" Columns:", list(tx.columns))
print("\n Purchase Behaviour Shape:", pb.shape)
print(" Columns:", list(pb.columns))


✅ Saved cleaned transaction data → data/cleaned/QVI_transaction_cleaned.csv
✅ Saved cleaned purchase behaviour data → data/cleaned/QVI_purchase_behaviour_cleaned.csv

📦 Transaction Data Shape: (264834, 12)
🧾 Columns: ['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR', 'PROD_NAME', 'PROD_QTY', 'TOT_SALES', 'BRAND', 'VARIANT_LINE', 'FLAVOR', 'PACK_SIZE']

👤 Purchase Behaviour Shape: (72637, 3)
🧠 Columns: ['LYLTY_CARD_NBR', 'LIFESTAGE', 'PREMIUM_CUSTOMER']


In [28]:
# --- Phase 3: Merge & Save Final Combined Dataset ---

# Load cleaned datasets
tx_clean = pd.read_csv("data/cleaned/QVI_transaction_cleaned.csv")
pb_clean = pd.read_csv("data/cleaned/QVI_purchase_behaviour_cleaned.csv")

# Merge datasets on customer ID
merged = pd.merge(tx_clean, pb_clean, on="LYLTY_CARD_NBR", how="inner")

# Check merge result
print(" Merge successful!")
print(f" Merged dataset shape: {merged.shape}")
print(f" Columns combined: {len(tx_clean.columns)} + {len(pb_clean.columns) - 1} → {len(merged.columns)} columns")

# Check for duplicates (each TXN_ID × PROD_NBR should be unique)
duplicates = merged.duplicated(subset=["TXN_ID", "PROD_NBR"]).sum()
print(f" Duplicate transaction-product pairs: {duplicates}")

# Save merged dataset to final folder
output_dir = "data/final"
os.makedirs(output_dir, exist_ok=True)
merged.to_csv(f"{output_dir}/QVI_merged_dataset.csv", index=False)

print("\n Final merged dataset saved as → data/final/QVI_merged_dataset.csv")

# Optional preview
print("\n Sample merged rows:")
print(merged.sample(5, random_state=42).to_string(index=False))


 Merge successful!
 Merged dataset shape: (264834, 14)
 Columns combined: 12 + 2 → 14 columns
 Duplicate transaction-product pairs: 1

 Final merged dataset saved as → data/final/QVI_merged_dataset.csv

 Sample merged rows:
      DATE  STORE_NBR  LYLTY_CARD_NBR  TXN_ID  PROD_NBR                                PROD_NAME  PROD_QTY  TOT_SALES     BRAND VARIANT_LINE                    FLAVOR  PACK_SIZE             LIFESTAGE PREMIUM_CUSTOMER
2019-05-23          7            7222    7214        88         Kettle Honey Soy    Chicken 175g         2       10.8    Kettle          NaN         Honey Soy Chicken      175.0 OLDER SINGLES/COUPLES           Budget
2019-03-15         10           10112    9712       104 Infuzions Thai SweetChili PotatoMix 110g         2        7.6 Infuzions          NaN Thai Sweetchili Potatomix      110.0        YOUNG FAMILIES          Premium
2019-06-19        166          166089  167582        90        Tostitos Smoked     Chipotle 175g         2        8.8  Tostit